In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import math as math
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn import preprocessing
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import pinv_extended
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

In [ ]:
pd.set_option('display.max_columns', None)
#reading in recomb rate intervals with features from ARG-inference
df = pd.read_csv('/home2/rke27/crossover_datasets/Maize_Teosinte_ARGinferred_v4_chrscombined_100kb_noscinot_phased_CG_CHG_tCG_tCHG_Mnase_H2az_H3K27me3_gene_H3K4me3_GCcontent2_H3K4me1_H3K9ac_H3K36me3_cent_bothtels.bed', sep='\t', header = None)
df.columns = ["chrom", "start", "end", "rate_maize", 'rate_teo', "maizeCG%", "maizeCHG%", "teoCG%", "teoCHG%","Mnase", "H2AZ", "H3K27me3", "gene_density", "H3K4me3", "GC_content", "H3K4me1","H3K9ac","H3K36me3", "dist_cent", "dist_tel"]
display(df)

In [ ]:
#changing these features to be float instead of an object
s = df.select_dtypes(include='object').columns
df[s] = df[s].astype("float")
#so recombination rate is whole numbers
df['rate_maize']=df['rate_maize']*1e8
df['rate_teo']=df['rate_teo']*1e8

In [ ]:
#dealing with missing data, usually replace as NaN so Python recognizes it
df.replace('NAN','NaN', regex = True, inplace = True)

In [ ]:
df["teoCG%"] = df["teoCG%"]*100
df["teoCHG%"] = df["teoCHG%"]*100

In [ ]:
#breaking up chromosomes into each file
df_chr1 = df[df['chrom'] == 1]
df_chr2 = df[df['chrom'] == 2]
df_chr3 = df[df['chrom'] == 3]
df_chr4 = df[df['chrom'] == 4]
df_chr5 = df[df['chrom'] == 5]
df_chr6 = df[df['chrom'] == 6]
df_chr7 = df[df['chrom'] == 7]
df_chr8 = df[df['chrom'] == 8]
df_chr9 = df[df['chrom'] == 9]
df_chr10 = df[df['chrom'] == 10]

In [ ]:
#scaling the telomere data to be centered & have a sd of 1
from sklearn.preprocessing import StandardScaler

scaler = MinMaxScaler()

# Fit and transform only the 'target_column'
df_chr1["dist_tel"] = scaler.fit_transform(df_chr1[["dist_tel"]])
df_chr2["dist_tel"] = scaler.fit_transform(df_chr2[["dist_tel"]])
df_chr3["dist_tel"] = scaler.fit_transform(df_chr3[["dist_tel"]])
df_chr4["dist_tel"] = scaler.fit_transform(df_chr4[["dist_tel"]])
df_chr5["dist_tel"] = scaler.fit_transform(df_chr5[["dist_tel"]])
df_chr6["dist_tel"] = scaler.fit_transform(df_chr6[["dist_tel"]])
df_chr7["dist_tel"] = scaler.fit_transform(df_chr7[["dist_tel"]])
df_chr8["dist_tel"] = scaler.fit_transform(df_chr8[["dist_tel"]])
df_chr9["dist_tel"] = scaler.fit_transform(df_chr9[["dist_tel"]])
df_chr10["dist_tel"] = scaler.fit_transform(df_chr10[["dist_tel"]])

In [ ]:
#scaling the centromere data to be centered & have a sd of 1
from sklearn.preprocessing import StandardScaler

scaler = MinMaxScaler()

# Fit and transform only the 'target_column'
df_chr1["dist_cent"] = scaler.fit_transform(df_chr1[["dist_cent"]])
df_chr2["dist_cent"] = scaler.fit_transform(df_chr2[["dist_cent"]])
df_chr3["dist_cent"] = scaler.fit_transform(df_chr3[["dist_cent"]])
df_chr4["dist_cent"] = scaler.fit_transform(df_chr4[["dist_cent"]])
df_chr5["dist_cent"] = scaler.fit_transform(df_chr5[["dist_cent"]])
df_chr6["dist_cent"] = scaler.fit_transform(df_chr6[["dist_cent"]])
df_chr7["dist_cent"] = scaler.fit_transform(df_chr7[["dist_cent"]])
df_chr8["dist_cent"] = scaler.fit_transform(df_chr8[["dist_cent"]])
df_chr9["dist_cent"] = scaler.fit_transform(df_chr9[["dist_cent"]])
df_chr10["dist_cent"] = scaler.fit_transform(df_chr10[["dist_cent"]])

In [ ]:
#Split the data differently, instead of training on chr. 1-8 & testing on chr. 9-10, we can just subset the data in the original dataframe that contained all chromosomes
df = pd.concat([df_chr1,df_chr3,df_chr4,df_chr5,df_chr6,df_chr7,df_chr8,df_chr9,df_chr10])

# Replacing infinite with nan 
df.replace([np.inf, -np.inf], np.nan, inplace=True) 

#centering all predictors by subtracting the mean to reduce overfitting
df['rate_maize'] = df['rate_maize']**0.5
df['rate_teo'] = df['rate_teo']**0.5
df['rate_maize'] = df['rate_maize'] - df['rate_maize'].mean()
df['rate_teo'] = df['rate_teo'] - df['rate_teo'].mean()
df['maizeCG%'] = df['maizeCG%'] - df['maizeCG%'].mean()
df['maizeCHG%'] = df['maizeCHG%'] - df['maizeCHG%'].mean()
df['teoCHG%'] = df['teoCHG%'] - df['teoCHG%'].mean()
df['teoCG%'] = df['teoCG%'] - df['teoCG%'].mean()
df['Mnase'] = df['Mnase'] - df['Mnase'].mean()
df['H2AZ'] = df['H2AZ'] - df['H2AZ'].mean()
df['H3K4me3'] = df['H3K4me3'] - df['H3K4me3'].mean()
df['H3K4me1'] = df['H3K4me1'] - df['H3K4me1'].mean()
df['H3K27me3'] = df['H3K27me3'] - df['H3K27me3'].mean()
df['dist_cent'] = df['dist_cent'] - df['dist_cent'].mean()
df['dist_tel'] = df['dist_tel'] - df['dist_tel'].mean()
df['GC_content'] = df['GC_content'] - df['GC_content'].mean()
df['gene_density'] = df['gene_density'] - df['gene_density'].mean()

#adding in interaction terms
df['distchrom'] = df['dist_cent'] * df['chrom']
df['maizeCGCHGint'] = df['maizeCG%'] * df['maizeCHG%']
df['maizeCGcentint'] = df['maizeCG%'] * df['dist_cent']
df['teoCGCHGint'] = df['teoCG%'] * df['teoCHG%']
df['teoCGcentint'] = df['teoCG%'] * df['dist_cent']
df['teoCHGcentint'] = df['teoCHG%'] * df['dist_cent']

# Dropping all the rows with nan values 
df.dropna(inplace=True)

# Printing df 
df

In [ ]:
#training maize full model
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold
import numpy as np

# Assuming you have your data in df
columns_of_interest = ['maizeCG%']
X = df[columns_of_interest].copy()
y = df['rate_maize'].copy()

# Initialize KFold with 10 splits
kf = KFold(n_splits=10, shuffle=True, random_state=10)

# Lists to store metrics
r2_scores = []
rmse_scores = []

# Perform 10-fold cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Fit the model on the training set
    model = sm.OLS(y_train, exog=X_train)
    
    res=model.fit_regularized(alpha=0.005, L1_wt=1)

    model_fit_regularized = model.fit(cov_type = "HC0",params=res.params) 
    pinv_wexog,_ = pinv_extended(model.wexog)
    normalized_cov_params = np.dot(pinv_wexog, np.transpose(pinv_wexog))
    summary = sm.regression.linear_model.OLSResults(model,res.params,normalized_cov_params)
    print(summary.summary())
    
    # Make predictions on the test set
    predictions = model_fit_regularized.predict(X_test)
    
    # Calculate R-squared and RMSE
    r2 = r2_score(y_test, predictions)
    rmse = mean_squared_error(y_test, predictions, squared=False)
    
    r2_scores.append(r2)
    rmse_scores.append(rmse)

    # Plotting for each fold (optional, you might want to plot only the final model)
    plt.scatter(y_test, predictions)
    plt.xlabel("Actual Recombination Rate")
    plt.ylabel("Predicted Recombination Rate")
    plt.title(f"Fold {len(r2_scores)}: Predicted vs Actual (Test Set)")

    # Add trendline
    z = np.polyfit(y_test, predictions, 1)
    p = np.poly1d(z)
    plt.plot(y_test, p(y_test), color='red')
    
    # Add R-squared value to plot
    plt.text(plt.xlim()[0], plt.ylim()[1], f'R-squared = {r2:.2f}', horizontalalignment='left', verticalalignment='top')
    plt.show()

# Calculate mean and standard deviation of R-squared and RMSE
mean_r2 = np.mean(r2_scores)
std_r2 = np.std(r2_scores)
mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)

print(f"Mean R-squared: {mean_r2:.2f} ± {std_r2:.2f}")
print(f"Mean RMSE: {mean_rmse:.9f} ± {std_rmse:.9f}")

In [ ]:
#printing just full model trained on entire maize dataset
model = sm.OLS(df['rate_maize'], exog=df[['maizeCG%']])
res=model.fit_regularized(alpha=0.005, L1_wt=1)
model_fit_regularized = model.fit(cov_type = "HC0",params=res.params) 
pinv_wexog,_ = pinv_extended(model.wexog)
normalized_cov_params = np.dot(pinv_wexog, np.transpose(pinv_wexog))
summary = sm.regression.linear_model.OLSResults(model,res.params,normalized_cov_params)
print(summary.summary())

In [ ]:
#training full model in teosinte

# Assuming you have your data in df
columns_of_interest =  ["teoCHG%"]
X = df[columns_of_interest].copy()
y = df['rate_teo'].copy()

# Initialize KFold with 10 splits
kf = KFold(n_splits=10, shuffle=True, random_state=1)

# Lists to store metrics
r2_scores = []
rmse_scores = []

# Perform 10-fold cross-validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Fit the model on the training set
    model = sm.OLS(y_train, exog=X_train)
    
    res=model.fit_regularized(alpha=0.05, L1_wt=1)

    model_fit_regularized = model.fit(cov_type = "HC0",params=res.params) 
    pinv_wexog,_ = pinv_extended(model.wexog)
    normalized_cov_params = np.dot(pinv_wexog, np.transpose(pinv_wexog))
    summary = sm.regression.linear_model.OLSResults(model,res.params,normalized_cov_params)
    print(summary.summary())

    # Make predictions on the test set
    predictions = model_fit_regularized.predict(X_test)
    
    # Calculate R-squared and RMSE
    r2 = r2_score(y_test, predictions)
    rmse = mean_squared_error(y_test, predictions, squared=False)
    
    r2_scores.append(r2)
    rmse_scores.append(rmse)

    # Plotting for each fold (optional, you might want to plot only the final model)
    plt.scatter(y_test, predictions)
    plt.xlabel("Actual Recombination Rate")
    plt.ylabel("Predicted Recombination Rate")
    plt.title(f"Fold {len(r2_scores)}: Predicted vs Actual (Test Set)")

    # Add trendline
    z = np.polyfit(y_test, predictions, 1)
    p = np.poly1d(z)
    plt.plot(y_test, p(y_test), color='red')
    
    # Add R-squared value to plot
    plt.text(plt.xlim()[0], plt.ylim()[1], f'R-squared = {r2:.2f}', horizontalalignment='left', verticalalignment='top')
    plt.show()

# Calculate mean and standard deviation of R-squared and RMSE
mean_r2 = np.mean(r2_scores)
std_r2 = np.std(r2_scores)
mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)

print(f"Mean R-squared: {mean_r2:.2f} ± {std_r2:.2f}")
print(f"Mean RMSE: {mean_rmse:.9f} ± {std_rmse:.9f}")

In [ ]:
#printing out teosinte full model
model = sm.OLS(df['rate_teo'], exog=df[['teoCHG%']])
res=model.fit_regularized(alpha=0.005, L1_wt=1)
model_fit_regularized = model.fit(cov_type = "HC0",params=res.params) 
pinv_wexog,_ = pinv_extended(model.wexog)
normalized_cov_params = np.dot(pinv_wexog, np.transpose(pinv_wexog))
summary = sm.regression.linear_model.OLSResults(model,res.params,normalized_cov_params)
print(summary.summary())

In [ ]:
#using simplified maize model to predict teosinte recombination rates
columns_of_interest = ['maizeCG%']
X_train = df[columns_of_interest].copy()
y_train = df['rate_maize'].copy()

X_test = df[columns_of_interest].copy()
y_test = df['rate_teo'].copy()

# Fit the model on the training set
model = sm.OLS(y_train, exog=X_train).fit(cov_type = "HC0")
print(model.summary())

# Make predictions on the test set
predictions = model.predict(X_test)

# Calculate R-squared and RMSE
r2 = r2_score(y_test, predictions)
rmse = mean_squared_error(y_test, predictions)

# Print the metrics
print(f"R-squared: {r2:.2f}")
print(f"RMSE: {rmse:.9f}")

# Plotting the results
plt.scatter(y_test, predictions)
plt.xlabel("Actual Recombination Rate")
plt.ylabel("Predicted Recombination Rate")
plt.title("Predicted vs Actual")

# Add trendline
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test, p(y_test), color='red')

# Add R-squared value to plot
plt.text(plt.xlim()[0], plt.ylim()[1], f'R-squared = {r2:.2f}', horizontalalignment='left', verticalalignment='top')
plt.savefig("Maize_prediction_inteosinte.pdf", format="pdf", bbox_inches='tight')
plt.show()

In [ ]:
#using simplified teosinte model to predict maize recombination rates
columns_of_interest = ['teoCG%']
X_train = df[columns_of_interest].copy()
y_train = df['rate_teo'].copy()

X_test = df[columns_of_interest].copy()
y_test = df['rate_maize'].copy()

# Fit the model on the training set
model = sm.OLS(y_train, exog=X_train).fit(cov_type = "HC0")
print(model.summary())
# Make predictions on the test set
predictions = model.predict(X_test)

# Calculate R-squared and RMSE
r2 = r2_score(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)

# Print the metrics
print(f"R-squared: {r2:.2f}")
print(f"RMSE: {rmse:.9f}")

# Plotting the results
plt.scatter(y_test, predictions)
plt.xlabel("Actual Recombination Rate")
plt.ylabel("Predicted Recombination Rate")
plt.title("Predicted vs Actual (Test Set)")

# Add trendline
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test, p(y_test), color='red')

# Add R-squared value to plot
plt.text(plt.xlim()[0], plt.ylim()[1], f'R-squared = {r2:.2f}', horizontalalignment='left', verticalalignment='top')
plt.savefig("Teosinte_prediction_inmaize.pdf", format="pdf", bbox_inches='tight')
plt.show()

In [ ]:
#correlating teosinte recombination landscape to maize recombination landscape with transformation & without
(df['rate_maize']**0.5).corr(df['rate_teo']**0.5, method = 'pearson')

In [ ]:
#correlating maize recombination landscape to gene density
(df['rate_maize']**0.5).corr(df['gene_density']**0.15, method = 'pearson')

In [ ]:
#correlating teosinte recombination landscape to gene density
(df['rate_teo']**0.5).corr(df['gene_density']**0.15, method = 'pearson')